In [1]:
from ib_insync import *
ib = IB()
util.startLoop()
ib.connect(port=4001,clientId=0)

<IB connected to 127.0.0.1:4001 clientId=0>

In [5]:
contract = Stock(symbol="TSLA",exchange="SMART",currency="USD")
ib.qualifyContracts(contract)
bars = ib.reqHistoricalData(contract,"",barSizeSetting='1 min',durationStr='3 M',whatToShow='TRADES',useRTH=False)

reqHistoricalData: Timeout for Stock(conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS')
Error 162, reqId 5327: Historical Market Data Service error message:API historical data query cancelled: 5327, contract: Stock(conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS')


In [13]:
def _make_df(bars):
    df = util.df(bars)
    df.set_index('date',inplace=True)
    df.index = df.index.tz_localize(None)
    return df 
import pandas as pd
import datetime
import pandas_market_calendars as mcal
import os
# Create a calendar
nyse = mcal.get_calendar('NYSE')
def get_historical_1m_data_fast(ib,symbol,start_date,end_date,what_to_show='TRADES',use_rth=False):    
    messup_count=0
    df_list=[]
    start_date = datetime.datetime.strptime(start_date,'%Y%m%d').date() # .date() is important to get correct date!
    start_date.strftime('%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date,'%Y%m%d').date()
    end_date.strftime('%Y-%m-%d')
    dates = nyse.schedule(start_date=start_date, end_date=end_date)
    dates = [t.strftime('%Y%m%d') for t in mcal.date_range(dates, frequency='1D').tolist()]
    dates = [datetime.datetime.strptime(t,'%Y%m%d').date() for t in dates]
    contract = Stock(symbol=symbol,exchange='SMART',currency='USD')
    ib.qualifyContracts(contract)

    last_end_date = dates[-1]+datetime.timedelta(days=1)
    for end_date in dates[::-1]:
        if end_date>=last_end_date: continue
        if messup_count>=2: break
        try:
            print("Retrieving data for ",symbol," for ",end_date)
            bars=ib.reqHistoricalData(contract,endDateTime=end_date,
                                                    barSizeSetting='1 min',whatToShow=what_to_show,useRTH=use_rth,
                                 durationStr='1 W')
            df=_make_df(bars)
            last_end_date=bars[0].date.date()
            df_list.append(df)
        except Exception as e:
            messup_count+=1
            print(e)
            print("No data for ",end_date)
        else:
            messup_count=0
    return pd.concat(df_list).sort_index()

In [14]:
df=get_historical_1m_data_fast(ib,'TSLA','20231101','20231201',what_to_show='TRADES',use_rth=False)

Retrieving data for  TSLA  for  2023-12-01
Retrieving data for  TSLA  for  2023-11-24
Retrieving data for  TSLA  for  2023-11-17
Retrieving data for  TSLA  for  2023-11-10
Retrieving data for  TSLA  for  2023-11-03


In [15]:
df

,open,high,low,close,volume,average,barCount
date,,,,,,,
2023-10-30 04:00:00,209.21,210.60,208.52,210.06,5142.0,209.630,22
2023-10-30 04:01:00,210.10,210.10,209.90,209.90,2456.0,210.016,16
2023-10-30 04:02:00,210.10,210.21,210.10,210.19,987.0,210.168,6
2023-10-30 04:03:00,210.05,210.05,209.76,209.86,1942.0,209.869,11
2023-10-30 04:04:00,209.80,209.80,209.80,209.80,200.0,209.800,1
...,...,...,...,...,...,...,...
2023-12-01 19:55:00,239.30,239.40,239.30,239.40,1682.0,239.310,10
2023-12-01 19:56:00,239.40,239.40,239.30,239.30,4223.0,239.355,5
2023-12-01 19:57:00,239.25,239.44,239.25,239.33,5120.0,239.339,17
